In [1]:
#update scikit-learn
!pip uninstall scikit-learn -y
!pip install scikit-learn
#!pip install scikit-multilearn
!pip install iterative-stratification
#!pip install sklearn-contrib-lightning
!pip install --upgrade numpy

Found existing installation: scikit-learn 1.0.1
Uninstalling scikit-learn-1.0.1:
  Successfully uninstalled scikit-learn-1.0.1
     |████████████████████████████████| 23.2 MB 67.3 MB/s 
     |████████████████████████████████| 15.7 MB 5.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 9.22 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 131 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 6.5 MB/s 
     |████████████████████████████████| 8.4 MB 53.2 MB/s 
     |████████████████████████████████| 138 kB 60.3 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
  Created wheel for bert-sklearn: filename

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Read in the cleaned data
data_prefix = 'https://raw.githubusercontent.com/nasa-petal/search-engine/main/data/'
df = pd.read_csv(data_prefix + 'cleaned.csv')

# convert array strings to actual arrays
from ast import literal_eval
df['y'] = df['y'].apply(literal_eval)
df.head()

,y,text
0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",building a home from foam tungara frog foam ne...
1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",a nocturnal mammal the greater mouse eared bat...
2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",polarization sensitivity in two species of cut...
3,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]",identification and characterization of a multi...
4,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",differences in polysaccharide structure betwee...


In [2]:
from tempfile import mkdtemp
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
#from lightning.classification import LinearSVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate
#from skmultilearn.model_selection import IterativeStratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

RANDOM_STATE = 42

tfidf = TfidfVectorizer(strip_accents="ascii", analyzer="char", ngram_range=(1, 5), max_features=500000)
scaler = MaxAbsScaler(copy=False)
estimator = Pipeline(
    [("tfidf", tfidf), ("scaler", scaler), ("svc", OneVsRestClassifier(LinearSVC(loss="squared_hinge", random_state=RANDOM_STATE)))],
    memory=mkdtemp(),
)
p_grid = {"svc__estimator__C": [5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1e0, 5e0, 1e1, 5e1]} # 

#inner_cv = IterativeStratification(n_splits=4, order=1)
#outer_cv = IterativeStratification(n_splits=4, order=1)
inner_cv = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
outer_cv = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
scoring = ['average_precision', 'precision_macro', 'recall_macro', 'f1_macro', 'accuracy', 'multilabel_confusion_matrix', 'classification_report']

# Nested CV with parameter optimization
clf = GridSearchCV(estimator=estimator, scoring="average_precision", param_grid=p_grid, cv=inner_cv)
nested_score = cross_validate(clf, X=df["text"], y=df["y"].tolist(), scoring="average_precision", cv=outer_cv)
print(f"The mean MAP score using nested cross-validation is: "
      f"{nested_score['test_score'].mean():.3f} +/- {nested_score['test_score'].std():.3f}")

The mean MAP score using nested cross-validation is: 0.627 +/- 0.018
